### Testing approach with SAM + Florence + Dino

In [ ]:
import sys, os
from pathlib import Path
from tqdm.auto import tqdm

# if using Apple MPS, fall back to CPU for unsupported ops
# os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from transformers import (
    AutoProcessor,
    AutoModelForCausalLM,
    AutoImageProcessor,
    AutoModel
)

# select the device for computation
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"using device: {device}")

if device.type == "cuda":
    # use bfloat16 for the entire notebook
    torch.autocast("cuda", dtype=torch.float16).__enter__()

In [ ]:
np.random.seed(0xDEADBEEF)

def show_anns(anns, borders=True):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:, :, 3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.5]])
        img[m] = color_mask 
        if borders:
            import cv2
            contours, _ = cv2.findContours(m.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
            # Try to smooth contours
            contours = [cv2.approxPolyDP(contour, epsilon=0.01, closed=True) for contour in contours]
            cv2.drawContours(img, contours, -1, (0, 0, 1, 0.4), thickness=1) 

    ax.imshow(img)

In [ ]:
def get_florence_caption(image_crop, model,
                         processor, device,
                         task="<CAPTION>"):
    """Получить caption от Florence-2"""
    inputs = processor(text=task, images=image_crop, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=1024,
        do_sample=False,
        num_beams=3,
        use_cache=False
    )

    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed = processor.post_process_generation(
        generated_text, task=task,
        image_size=(image_crop.width, image_crop.height)
    )

    return parsed[task]


def get_dinov2_embedding(image_crop, model,
                         processor, device):
    """Получить CLS token от DINOv2"""
    inputs = processor(images=image_crop, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    return outputs.last_hidden_state[0, 0].cpu().numpy()


def get_dinov2_embeddings_batch(image_crops, model,
                                processor, device, batch_size=8):
    """Получить эмбеддинги батчами (заготовка для оптимизации)"""
    embeddings = []

    for i in range(0, len(image_crops), batch_size):
        batch = image_crops[i:i + batch_size]
        inputs = processor(images=batch, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = model(**inputs)

        # CLS tokens для всего батча
        batch_embeddings = outputs.last_hidden_state[:, 0].cpu().numpy()
        embeddings.extend(batch_embeddings)

    return embeddings


def process_masks_with_features(image, masks,
                                florence_model, florence_processor, florence_device,
                                dino_model, dino_processor, dino_device,
                                use_batch=False, batch_size=8):
    """Обработать маски: добавить description и embedding"""

    # Собираем все crops
    crops = []
    for mask_data in masks:
        x, y, w, h = mask_data['bbox']
        crop = image.crop((x, y, x + w, y + h))
        crops.append(crop)

    # Florence captions (пока без батчинга)
    print("Generating captions...")
    for idx, (mask_data, crop) in enumerate(tqdm(zip(masks, crops), total=len(masks))):
        mask_data['description'] = get_florence_caption(
            crop, florence_model, florence_processor, florence_device
        )

    # DINOv2 embeddings (с опциональным батчингом)
    print("Extracting embeddings...")
    if use_batch:
        embeddings = get_dinov2_embeddings_batch(
            crops, dino_model, dino_processor, dino_device, batch_size
        )
        for mask_data, embedding in zip(masks, embeddings):
            mask_data['embedding'] = embedding
    else:
        for mask_data, crop in tqdm(zip(masks, crops), total=len(masks)):
            mask_data['embedding'] = get_dinov2_embedding(
                crop, dino_model, dino_processor, dino_device
            )

    return masks

def visualize_masks_with_descriptions(image, masks, figsize=(25, 20)):
    """Визуализация масок с номерами и списком описаний"""
    fig, (ax_img, ax_text) = plt.subplots(1, 2, figsize=figsize,
                                            gridspec_kw={'width_ratios': [3, 1]})

    # Изображение с масками и номерами
    ax_img.imshow(image)
    plt.sca(ax_img)
    show_anns(masks)

    # Добавляем номера в центре каждой маски
    for idx, mask_data in enumerate(masks):
        x, y, w, h = mask_data['bbox']
        center_x = x + w / 2
        center_y = y + h / 2

        ax_img.text(center_x, center_y, str(idx),
                    color='white', fontsize=14, weight='bold',
                    ha='center', va='center',
                    bbox=dict(boxstyle='round,pad=0.5', facecolor='black', alpha=0.6))

    ax_img.axis('off')

    # Список описаний
    ax_text.axis('off')
    descriptions = "\n\n".join([f"{i}: {mask['description']}" for i, mask in enumerate(masks)])
    ax_text.text(0.05, 0.50, descriptions,
                fontsize=11, va='center', ha='left',
                family='monospace', wrap=True)

    plt.tight_layout()
    plt.show()

### Baseline

In [ ]:
projectDir = Path.cwd().parent
dataDir = projectDir / "data/scand_spot_fountain-lib/output_frames"

frames = sorted(dataDir.glob("frame_*.png"))

### SAM 1

In [ ]:
sys.path.insert(0, str(projectDir / "segment-anything"))
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "/mnt/vol0/weights/sam/sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=4,
    pred_iou_thresh=0.95,
    )

### SAM 2

In [ ]:
# sys.path.insert(0, str(projectDir / "sam2"))
# from sam2.build_sam import build_sam2
# from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator

# sam2_checkpoint = str(projectDir / "sam2/checkpoints/sam2.1_hiera_large.pt")
# model_cfg = 'configs/sam2.1/sam2.1_hiera_l.yaml'

# sam2 = build_sam2(model_cfg, sam2_checkpoint, device=device, apply_postprocessing=False)
# mask_generator = SAM2AutomaticMaskGenerator(sam2)

In [ ]:
torch_dtype = torch.float16

fl_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Florence-2-large-ft",
    dtype=torch_dtype,
    trust_remote_code=True,
    attn_implementation="eager"
).to(device)

fl_processor = AutoProcessor.from_pretrained(
    "microsoft/Florence-2-large-ft",
    trust_remote_code=True
)

dino_model = AutoModel.from_pretrained("facebook/dinov2-base").to(device)
dino_model.eval()
dino_processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base", use_fast=True)


In [ ]:
mask_db = []

for index, frame in enumerate(frames):
    if index == 0: continue
    print(f"Frame {index}: {frame}")
    image = Image.open(frame).convert("RGB")

    plt.figure(figsize=(12, 12))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

    masks = mask_generator.generate(np.array(image))

    plt.figure(figsize=(12, 12))
    plt.imshow(image)
    show_anns(masks)
    plt.axis('off')
    plt.show() 

    masks = process_masks_with_features(
        image, masks,
        fl_model, fl_processor, device,
        dino_model, dino_processor, device
    )

    visualize_masks_with_descriptions(image, masks)
    mask_db.append((frame.name, masks))
    
    if index == 2:
        break

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from umap import UMAP

# 1. Берем только первые 2 кадра
frame0_embeddings = np.array([mask['embedding'] for mask in mask_db[0][1]])
frame1_embeddings = np.array([mask['embedding'] for mask in mask_db[1][1]])

# Объединяем
all_embeddings = np.vstack([frame0_embeddings, frame1_embeddings])

# Метки для цветов (0 = frame 0, 1 = frame 1)
labels = np.array([0] * len(frame0_embeddings) + [1] * len(frame1_embeddings))
colors = ['#FF6B6B', '#4ECDC4']  # красный для frame 0, бирюзовый для frame 1

print(f"Frame 0: {len(frame0_embeddings)} objects")
print(f"Frame 1: {len(frame1_embeddings)} objects")
print(f"Total: {len(all_embeddings)} objects, dim: {all_embeddings.shape[1]}")

# 2. Применяем редукцию размерности
pca = PCA(n_components=2)
tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(all_embeddings)-1))
umap = UMAP(n_components=2, random_state=42)

embeddings_pca = pca.fit_transform(all_embeddings)
embeddings_tsne = tsne.fit_transform(all_embeddings)
embeddings_umap = umap.fit_transform(all_embeddings)

# 3. Визуализация
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

methods = [
    (embeddings_pca, 'PCA', f'explained variance: {pca.explained_variance_ratio_.sum():.2%}'),
    (embeddings_tsne, 't-SNE', ''),
    (embeddings_umap, 'UMAP', '')
]

for ax, (embeddings_2d, title, subtitle) in zip(axes, methods):
    # Frame 0
    ax.scatter(embeddings_2d[labels==0, 0], embeddings_2d[labels==0, 1],
                c=colors[0], label='Frame 0', alpha=0.7, s=100, edgecolors='black')
    # Frame 1
    ax.scatter(embeddings_2d[labels==1, 0], embeddings_2d[labels==1, 1],
                c=colors[1], label='Frame 1', alpha=0.7, s=100, edgecolors='black')

    ax.set_title(f'{title}\n{subtitle}', fontsize=14, weight='bold')
    ax.legend()
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Дополнительно - аннотировать номера объектов:
# # Добавить номера объектов на график
# for i, (x, y) in enumerate(embeddings_2d):
#     ax.annotate(str(i), (x, y), fontsize=8, ha='center', va='center')

In [ ]:
# Жадный алгоритм для matching объектов между кадрами
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

def greedy_match_objects(frame0_embeddings, frame1_embeddings, threshold=0.7):
    """Жадное сопоставление объектов по косинусному расстоянию"""
    sim_matrix = cosine_similarity(frame0_embeddings, frame1_embeddings)

    matches = []
    used_frame1 = set()

    # Для каждого объекта из frame0
    for i in range(len(frame0_embeddings)):
        best_j = None
        best_sim = threshold

        for j in range(len(frame1_embeddings)):
            if j not in used_frame1 and sim_matrix[i, j] > best_sim:
                best_sim = sim_matrix[i, j]
                best_j = j

        if best_j is not None:
            matches.append((i, best_j, best_sim))
            used_frame1.add(best_j)

    return matches

def visualize_object_tracking(image0, masks0, image1, masks1, matches):
    """Визуализация перемещения объектов со стрелками"""
    fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(20, 10))

    # Frame 0
    ax0.imshow(image0)
    ax0.set_title('Frame 0', fontsize=16)
    ax0.axis('off')

    # Frame 1
    ax1.imshow(image1)
    ax1.set_title('Frame 1', fontsize=16)
    ax1.axis('off')

    # Рисуем соответствия
    colors = plt.cm.rainbow(np.linspace(0, 1, len(matches)))

    for (i, j, sim), color in zip(matches, colors):
        # Центры bbox
        x0, y0, w0, h0 = masks0[i]['bbox']
        x1, y1, w1, h1 = masks1[j]['bbox']

        center0 = (x0 + w0/2, y0 + h0/2)
        center1 = (x1 + w1/2, y1 + h1/2)

        # Bbox на обоих кадрах
        rect0 = mpatches.Rectangle((x0, y0), w0, h0,
                                    linewidth=3, edgecolor=color, facecolor='none')
        rect1 = mpatches.Rectangle((x1, y1), w1, h1,
                                    linewidth=3, edgecolor=color, facecolor='none')
        ax0.add_patch(rect0)
        ax1.add_patch(rect1)

        # Номера и similarity
        ax0.text(center0[0], center0[1], str(i),
                color='white', fontsize=14, weight='bold',
                ha='center', va='center',
                bbox=dict(boxstyle='round', facecolor=color, alpha=0.8))

        ax1.text(center1[0], center1[1], f"{j}\n{sim:.2f}",
                color='white', fontsize=12, weight='bold',
                ha='center', va='center',
                bbox=dict(boxstyle='round', facecolor=color, alpha=0.8))

    plt.tight_layout()
    plt.show()

    return matches

# Использование:
frame0_embeddings = np.array([mask['embedding'] for mask in mask_db[0][1]])
frame1_embeddings = np.array([mask['embedding'] for mask in mask_db[1][1]])

matches = greedy_match_objects(frame0_embeddings, frame1_embeddings, threshold=0.7)

print(f"Found {len(matches)} matches:")
for i, j, sim in matches:
    print(f"  Frame0[{i}] -> Frame1[{j}]: {sim:.3f}")
    print(f"    '{mask_db[0][1][i]['description']}' -> '{mask_db[1][1][j]['description']}'")

# Визуализация
image0 = Image.open(frames[1]).convert("RGB")
image1 = Image.open(frames[2]).convert("RGB")

visualize_object_tracking(image0, mask_db[0][1], image1, mask_db[1][1], matches)